In [1]:
from transformers import ElectraForPreTraining, ElectraTokenizerFast
from transformers import pipeline
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import pandas as pd


model = "Seznam/small-e-czech"
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline(task="text-classification", model=model)

Some weights of the model checkpoint at Seznam/small-e-czech were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at Seznam/small-e-czech and are newly initialized: ['classifier.out_proj.weight', 'classifi

In [84]:
# df = pd.read_csv('data/sample_data.csv')
# df_train, df_test = train_test_split(df, test_size=0.33, random_state=42, shuffle=True)

# df_train.to_csv('data/sample_data_train.csv', index=False)
# df_test.to_csv('data/sample_data_test.csv', index=False)

df_train = pd.read_csv('data/sample_data_train.csv')
df_test = pd.read_csv('data/sample_data_test.csv')

df_train.head()

,text,label
0,lachtan,1
1,dům,0
2,liška,1
3,strom,0
4,matematika,0


In [85]:
from datasets import load_dataset, ClassLabel
import nlp
ds = load_dataset('csv', data_files={'train': 'data/sample_data_train.csv', 'test': 'data/sample_data_test.csv'})
label_features = ClassLabel(names=["není zvíře", "je zvíře"])
train_ds = ds['train']
# train_ds['label'] = label_features
train_ds.cast_column("label", label_features)
test_ds = ds['test']

Using custom data configuration default-f8ec23a20551b00e
Found cached dataset csv (C:/Users/capek/.cache/huggingface/datasets/csv/default-f8ec23a20551b00e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\capek\.cache\huggingface\datasets\csv\default-f8ec23a20551b00e\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-ea30b3ae017bf277.arrow


In [89]:
ds = train_ds
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 11
})

In [91]:
features = ds.features.copy()
features["label"] = ClassLabel(names=["není zvíře", "je zvíře"])
def adjust_labels(batch):
#     batch["label"] = [sentiment + 1 for sentiment in batch["label"]]
    return batch
ds = ds.map(adjust_labels, batched=True, features=features)
ds

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 11
})

In [93]:
ds.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['není zvíře', 'je zvíře'], id=None)}

In [95]:
print(ds.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['není zvíře', 'je zvíře'], id=None)}


In [53]:
# train_ds.features["basic_sentiment"] = ClassLabel(names=["negative", "neutral", "positive"])

In [54]:
print(train_ds.features)

{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [55]:
print(train_ds[0])

{'text': 'lachtan', 'label': 1}


In [97]:
import pandas as pd

ds.set_format(type="pandas")
df = ds[:]
df.head()

,text,label
0,lachtan,1
1,dům,0
2,liška,1
3,strom,0
4,matematika,0


In [56]:
tf_train = ds['train'].to_tf_dataset(
    columns=["text"],
    label_cols=["label"],
    batch_size=4,
    shuffle=True)

tf_test = ds['test'].to_tf_dataset(
    columns=["text"],
    label_cols=["label"],
    batch_size=4,
    shuffle=True)

In [57]:
tf_test

<PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [15]:
ds['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [4]:
train_ds.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [4]:
tokenized_data_train = tokenizer(train_ds['text'], return_tensors="tf", padding=True, truncation=True)
labels_train = np.array(train_ds['target'])
tokenized_data_test = tokenizer(test_ds['text'], return_tensors="tf", padding=True, truncation=True)
labels_test = np.array(test_ds['target'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
tokenized_data_train

{'input_ids': <tf.Tensor: shape=(11, 5), dtype=int32, numpy=
array([[    2, 16102,   154, 10097,     3],
       [    2,  1971,     3,     0,     0],
       [    2, 20550,     3,     0,     0],
       [    2,  6305,     3,     0,     0],
       [    2, 16537,     3,     0,     0],
       [    2, 22251,     3,     0,     0],
       [    2, 17998,   187,     3,     0],
       [    2,  7733,     3,     0,     0],
       [    2,  3876,     3,     0,     0],
       [    2, 18040,     3,     0,     0],
       [    2, 15483,   246,     3,     0]])>, 'token_type_ids': <tf.Tensor: shape=(11, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(11, 5), dtype=int32, numpy=
array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 0],
       

In [6]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained(model)
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))

model.fit(tokenized_data_train, labels_train)

Some layers from the model checkpoint at Seznam/small-e-czech were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at Seznam/small-e-czech and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss comp

ValueError: Cannot generate a hashable key for IteratorSpec(({'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 5), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None)),) because the _serialize() method returned an unsupproted value of type <class 'transformers.tokenization_utils_base.BatchEncoding'>

In [13]:
model.predict(tokenized_data_test)

1/1 [==============================] - 0s 22ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[0.05621602],
       [0.05620134],
       [0.06481404],
       [0.05680045],
       [0.0567313 ],
       [0.05758224]], dtype=float32), hidden_states=None, attentions=None)

In [134]:
X_train = df_train.text.apply(tokenize_text).values
X_test = np.asarray(df_test.text.apply(tokenize_text))

In [135]:
X_train

array([array([    2, 16102,   154, 10097,     3]),
       array([   2, 1971,    3]), array([    2, 20550,     3]),
       array([   2, 6305,    3]), array([    2, 16537,     3]),
       array([    2, 22251,     3]), array([    2, 17998,   187,     3]),
       array([   2, 7733,    3]), array([   2, 3876,    3]),
       array([    2, 18040,     3]), array([    2, 15483,   246,     3])],
      dtype=object)

In [128]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model, num_labels=1)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...classifier
......vars
...classifier\dense
......vars
.........0
.........1
...classifier\dropout
......vars
...classifier\out_proj
......vars
.........0
.........1
...electra
......vars
...electra\embeddings
......vars
.........0
.........1
.........2
...electra\embeddings\LayerNorm
......vars
.........0
.........1
...electra\embeddings\dropout
......vars
...electra\embeddings_project
......vars
.........0
.........1
...electra\encoder
......vars
...electra\encoder\layer\tf_electra_layer
......vars
...electra\encoder\layer\tf_electra_layer\attention
......vars
...electra\encoder\layer\tf_electra_layer\attention\dense_output
......vars
...electra\encoder\layer\tf_electra_layer\attention\dense_output\LayerNorm
......vars
.........0
.........1
...electra\encoder\layer\tf_electra_layer\attention\dense_output\dense
......vars
.........0
.........1
...electra\encoder\layer\tf_electra_layer\attention\dense_output\dropout
...

C:\Users\capek\pycharm_projects\Transformers_for_czech_language\venv\lib\site-packages\transformers\generation\tf_utils.py:446: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


File Name                                             Modified             Size
config.json                                    2022-12-22 11:42:54         2143
metadata.json                                  2022-12-22 11:42:54           64
variables.h5                                   2022-12-22 11:42:54     54702448
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-12-22 11:42:54         2143
metadata.json                                  2022-12-22 11:42:54           64
variables.h5                                   2022-12-22 11:42:54     54702448
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...classifier
......vars
...classifier\dense
......vars
.........0
.........1
...classifier\dropout
......vars
...classifier\out_proj
......vars
.........0
.........1
...electra
......vars
...electra\embeddings
......vars
.........0
.........1
.........2
...electra

PermissionDeniedError: Failed to delete a file: C:\Users\capek\AppData\Local\Temp\tmp2agw6zos/variables.h5; Permission denied

In [130]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(3e-5))
model.fit(np.array(df.text.apply(tokenize_text)), np.array(df.target))

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [129]:
df_test

,text,target
0,Kočka,1
1,pes,1
5,velbloud,1
15,atom,0
11,počítač,0
14,kalkulačka,0
